In [9]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [36]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    message_sentiment: str

graph = StateGraph(State)

In [37]:
def sentiment_node(state: State) -> State:
    return {
        "message_sentiment": detect_sentiment(state["messages"][-1]),
    }

def complaints_handler_node(state: State) -> State:
    return {
        "messages": [
            "Complaint registered.",
        ],
    }

def support_handler_node(state: State) -> State:
    return {
        "messages": [
            "How can I help you?",
        ],
    }

def detect_sentiment(message: str) -> str:
    return 'unhappy'



def conditional_edge(state: State) -> Literal['complaints', 'support']:
    if state["message_sentiment"] == 'unhappy':
        return "complaints"
    else:
        return "support"


graph.add_node(sentiment_node, "sentiment")
graph.add_node(complaints_handler_node, "complaints")
graph.add_node(support_handler_node, "support")

graph.add_conditional_edges(
    '__start__',
    conditional_edge,
    {
        'complaints': '__end__',
        'support': '__end__',
    }
)

In [38]:



graph.add_edge("complaints", "__end__")
graph.add_edge("support", "__end__")

In [39]:

graph.set_entry_point("__start__")
graph.compile()


graph.invoke({"messages": ["I'm unhappy with the service."]})



ValueError: START cannot be an end node